# Grid Radar -> Plot Max-CAPPI

We are using [fast-barnes-py](https://github.com/MeteoSwiss/fast-barnes-py.git) to grid the radar data. Please cite if you use it in your research.

In [ ]:
import fsspec
import radarx as rx
import xradar as xd
import cmweather  # noqa
import matplotlib.pyplot as plt

In [ ]:
file = "s3://noaa-nexrad-level2/2022/03/30/KGWX/KGWX20220330_234639_V06"
local_file = fsspec.open_local(
    f"simplecache::s3://{file}",
    s3={"anon": True},
    filecache={"cache_storage": "."},
)
dtree = xd.io.open_nexradlevel2_datatree(local_file)
dtree = rx.combine_nexrad_sweeps(dtree)
dtree = dtree.xradar.georeference()

In [ ]:
dtree.groups

In [ ]:
def filter_radar(ds):
    ds = ds.where((ds.DBZH > -10) & (ds.DBZH < 75))
    return ds

In [ ]:
dtree = dtree.xradar.map_over_sweeps(filter_radar)

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots(figsize=(7, 5))
dtree["sweep_2"]["DBZH"].plot.contourf(
    x="x",
    y="y",
    levels=range(-10, 75),
    cmap="NWSRef",
    ylim=(-200e3, 300e3),  # Adjust y-axis limits
    xlim=(-200e3, 300e3),  # Adjust x-axis limits
    ax=ax,  # Use the created axis
)
# Set the title
ax.set_title(
    f"{dtree.attrs['instrument_name']} {dtree['sweep_0']['time'].min().values}"
)
# Show the plot
plt.show()

In [ ]:
%%time
ds = dtree.radarx.to_grid(
    data_vars=["DBZH"],
    pseudo_cappi=True,
    x_lim=(-100000.0, 100000.0),
    y_lim=(-100000.0, 100000.0),
    z_lim=(0, 10000.0),
    x_step=1000,
    y_step=1000,
    z_step=250,
    x_smth=0.2,
    y_smth=0.2,
    z_smth=1,
)

In [ ]:
ds.radarx.plot_max_cappi("DBZH", cmap="ChaseSpectral", add_slogan=True)

In [ ]:
%%time
ds2 = dtree.radarx.to_grid(
    data_vars=["DBZH"],
    pseudo_cappi=False,
    x_lim=(-200000.0, 200000.0),
    y_lim=(-200000.0, 200000.0),
    z_lim=(0, 15000.0),
    x_step=1000,
    y_step=1000,
    z_step=250,
    x_smth=0.2,
    y_smth=0.2,
    z_smth=1,
)

In [ ]:
ds2.radarx.plot_max_cappi("DBZH", cmap="ChaseSpectral", add_slogan=True)